<a href="https://colab.research.google.com/github/mftnakrsu/ai-competition/blob/main/aicompetition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
import pandas as pd
import numpy as np
import zipfile
import os
import cv2
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.applications import ResNet50
from keras.utils import to_categorical
from tensorflow.keras import layers, models, regularizers  # Import regularizers
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
# Modeli tanımlama (Daha önceki EfficientNetV2 kodunu kullanabilirsiniz).
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers, models
import os
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

# Unzip

In [4]:
# Yüklediğiniz zip dosyasının yolunu buraya yazın
zip_file_path = '/content/drive/MyDrive/bosch_ai_competition/train.zip'  # Örneğin dosyanızın adı 'dosya.zip' ise

In [8]:
# !unzip '/content/drive/MyDrive/bosch_ai_competition/train.zip'


# Preprocessing

In [11]:
import os
os.getcwd()

'/content'

In [19]:
train_data_path = "/content/drive/MyDrive/bosch_ai_competition/train_labels.csv"
train_data = pd.read_csv(train_data_path)
train_data['image_name'] = train_data['image_name'].astype(str)  # Metne dönüştür
images = []
labels = []

In [28]:
for index, row in train_data.iterrows():
    # Dosya yolunu oluştururken .png uzantısını ekleyin
    img_path = os.path.join("/content/drive/MyDrive/bosch_ai_competition/train", f"{row['image_name']}.png")

    if not os.path.exists(img_path):
        print(f"Dosya bulunamadı: {img_path}")  # Hata mesajı
    else:
        img = cv2.imread(img_path)
        if img is None:
            print(f"Resim açılamadı: {img_path}")  # Resim açılmadıysa hata mesajı
        else:
            img = cv2.resize(img, (32, 32))  # Resmi yeniden boyutlandırma
            images.append(img)
            labels.append(row['label'])

# Listeleri NumPy dizilerine dönüştürün
images = np.array(images)
labels = np.array(labels)

print(f"Toplam {len(images)} resim işlendi.")


Resim açılamadı: /content/drive/MyDrive/bosch_ai_competition/train/10224.png
Toplam 24999 resim işlendi.


In [37]:
# images[1], labels[1] debugging

# Encoding

In [38]:
# Etiketleri Sayısal Formata Çevirme
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)  # String -> Integer
num_classes = len(label_encoder.classes_)  # Sınıf sayısı

# One-Hot Encoding
labels_one_hot = to_categorical(labels_encoded, num_classes)

X_train, X_val, y_train, y_val = train_test_split(images, labels_one_hot, test_size=0.2, random_state=42)


In [42]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(4).shuffle(buffer_size=1000)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(4)

In [43]:
# Veri Augmentasyonu için fonksiyon
def augment(image, label):
    seed = tf.random.uniform([2], minval=0, maxval=int(1e6), dtype=tf.int32)
    # Yatay çevirme
    image = tf.image.stateless_random_flip_left_right(image, seed=seed)
    # Parlaklık değişimi
    image = tf.image.stateless_random_brightness(image, max_delta=0.2, seed=seed)
    # Kontrast değişimi
    image = tf.image.stateless_random_contrast(image, lower=0.8, upper=1.2, seed=seed)
    # Kırpma ve yeniden boyutlandırma
    image = tf.image.resize_with_crop_or_pad(image, 48, 48)
    image = tf.image.random_crop(image, size=[32, 32, 3])
    # Yaklaştırma/uzaklaştırma
    scales = tf.random.uniform([], minval=0.9, maxval=1.1)
    new_height = tf.cast(32 * scales, tf.int32)
    new_width = tf.cast(32 * scales, tf.int32)
    image = tf.image.resize(image, [new_height, new_width])
    image = tf.image.resize_with_crop_or_pad(image, 32, 32)

    return image, label

In [44]:
# Dataset Pipeline
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = (
    train_dataset
    .shuffle(buffer_size=1000)
    .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(4)
    .prefetch(tf.data.AUTOTUNE)
)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(4).prefetch(tf.data.AUTOTUNE)

In [61]:
num_classes=50
# TPU kullanarak model tanımlama
# Önceden eğitilmiş ConvNeXt modelini yükle
base_model = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    weights='imagenet',
    input_shape=(32, 32, 3),
)

base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False

In [62]:
# Modeli oluştur
model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),

    layers.Dense(num_classes, activation='softmax')
])

# Model Derleme
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [63]:
num_classes

50

In [64]:
# Modeli TPU üzerinde eğit
history = model.fit(
train_dataset,
validation_data=val_dataset,
epochs=10000,
callbacks=[early_stopping, reduce_lr]
)

Epoch 1/10000
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 196s 27ms/step - accuracy: 0.0208 - loss: 5.2518 - val_accuracy: 0.0196 - val_loss: 3.9706 - learning_rate: 0.0010
Epoch 2/10000
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 127s 18ms/step - accuracy: 0.0189 - loss: 3.9556 - val_accuracy: 0.0180 - val_loss: 3.9137 - learning_rate: 0.0010
Epoch 3/10000
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 98s 20ms/step - accuracy: 0.0176 - loss: 3.9133 - val_accuracy: 0.0180 - val_loss: 3.9137 - learning_rate: 0.0010
Epoch 4/10000
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 138s 19ms/step - accuracy: 0.0174 - loss: 3.9133 - val_accuracy: 0.0180 - val_loss: 3.9138 - learning_rate: 0.0010
Epoch 5/10000
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 140s 19ms/step - accuracy: 0.0173 - loss: 3.9129 - val_accuracy: 0.0180 - val_loss: 3.9137 - learning_rate: 5.0000e-04
Epoch 6/10000
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 93s 18ms/step - accuracy: 0.0183 - loss: 3.9126 - val_accuracy: 0.0180 - val_loss: 3.9137 - learning_rate: 5.0000e-04
Epoch 7/10000
5000/5000 ━━━━━━━━━━

KeyboardInterrupt: 